In [1]:
import pandas as pd
from datetime import datetime, timedelta
from unidecode import unidecode
import utils

def trim_overspace(text):
	parts = filter(lambda x: len(x) > 0,text.split(" "))
	return " ".join(parts)

def normalize_text(text):
    #.replace("'"," ") alguns municipios tem ' e as macros maravilhosas do excel vão falhar
	x = str(text).replace(".","").replace("\n","").replace(",","").upper()
	x = trim_overspace(x)
	x = unidecode(x)

	if x == 'nan' or len(x) == 0 or x == '0' or x == 'nao_informado':
		return ''
	else:
		return x
    
def normalize_labels(text):
	x = str(text).replace("'"," ").replace(".","").replace("\n","").replace(",","").lower()
	x = trim_overspace(x).replace(" ","_")
	x = unidecode(x)

	if x == 'nan' or len(x) == 0 or x == '0' or x == 'nao_informado':
		return ''
	else:
		return x
    
def normalize_number(num,cast=int,error='fill',fill='0'):
	try:
		return cast(num)
	except ValueError:
		if error == 'raise':
			raise Exception(ValueError)
		elif error == 'fill':
			return normalize_number(fill,cast,'raise')

In [2]:
# notifica = pd.read_csv('query.csv', converters={''} parse_dates=['data_notificacao','data_liberacao','data_1o_sintomas','data_cura_obito'], date_parser=lambda x: pd.to_datetime(x,format='%d/%m/%Y'))
notifica = pd.read_csv('query.csv',
                       converters = {
                           'paciente': normalize_text,
                           'idade': lambda x: normalize_number(x,fill=-1),
                           'ibge_residencia': lambda x: normalize_number(x,fill=-1),
                           'ibge_unidade_notifica': lambda x: normalize_number(x,fill=-1),                           
                           'exame': lambda x: normalize_number(x,fill=0),
                           'evolucao': lambda x: normalize_number(x,fill=3)
                       },
                       parse_dates = ['data_notificacao','updated_at','data_liberacao','data_1o_sintomas','data_cura_obito'],
                       date_parser = lambda x: pd.to_datetime(x, errors='coerce', format='%d/%m/%Y')
                      )
notifica.dtypes

id                                int64
data_notificacao         datetime64[ns]
updated_at               datetime64[ns]
paciente                         object
sexo                             object
idade                             int64
ibge_residencia                   int64
ibge_unidade_notifica             int64
exame                             int64
data_liberacao           datetime64[ns]
data_1o_sintomas         datetime64[ns]
evolucao                          int64
data_cura_obito          datetime64[ns]
dtype: object

In [3]:
hoje = datetime.today()
ontem = hoje - timedelta(1)
anteontem = ontem - timedelta(1)

In [4]:
municipios = utils.municipios[['ibge','municipio']].copy()
municipios['municipio'] = municipios['municipio'].apply(normalize_text)

regionais = utils.regionais[['ibge','nu_reg']].copy()
regionais = regionais.rename(columns={'ibge':'ibge_residencia','nu_reg':'rs'})

exames = utils.termos.loc[utils.termos['tipo']=='exame',['codigo','valor']].copy()
exames = exames.rename(columns={'codigo':'exame','valor':'nome_exame'})

In [5]:
municipios = municipios.rename(columns={'ibge':'ibge_residencia','municipio':'mun_resid'})
notifica = pd.merge(left=notifica, right=municipios, how='left', on='ibge_residencia')
notifica = pd.merge(left=notifica, right=regionais, how='left', on='ibge_residencia')

municipios = municipios.rename(columns={'ibge_residencia':'ibge_unidade_notifica','mun_resid':'mun_atend'})
notifica = pd.merge(left=notifica, right=municipios, how='left', on='ibge_unidade_notifica')
notifica = pd.merge(left=notifica, right=exames, how='left', on='exame')
notifica.dtypes

id                                int64
data_notificacao         datetime64[ns]
updated_at               datetime64[ns]
paciente                         object
sexo                             object
idade                             int64
ibge_residencia                   int64
ibge_unidade_notifica             int64
exame                             int64
data_liberacao           datetime64[ns]
data_1o_sintomas         datetime64[ns]
evolucao                          int64
data_cura_obito          datetime64[ns]
mun_resid                        object
rs                              float64
mun_atend                        object
nome_exame                       object
dtype: object

In [6]:
notifica['rs'] = notifica['rs'].apply(lambda x: normalize_number(x,fill='99'))
notifica['rs'] = notifica['rs'].apply(lambda x: str(x).zfill(2) if x != 99 else None)
notifica = notifica[['id','updated_at','data_notificacao','paciente','sexo','idade','mun_resid','mun_atend','rs','nome_exame','data_liberacao','data_1o_sintomas','evolucao','data_cura_obito']].copy()
notifica.shape

(212970, 14)

In [7]:
casos_novos_raw = notifica.loc[(notifica['evolucao'] == 3) & ((notifica['updated_at'] >= anteontem) | (notifica['data_liberacao'] >= anteontem) | (notifica['data_notificacao'] >= anteontem))]
print(casos_novos_raw.shape)
obitos_novos_raw = notifica.loc[(notifica['evolucao'] == 2) & ((notifica['updated_at'] >= anteontem) | (notifica['data_liberacao'] >= anteontem) | (notifica['data_notificacao'] >= anteontem))]
print(obitos_novos_raw.shape)

(3622, 14)
(67, 14)


In [8]:
casos_novos_raw = casos_novos_raw[['id', 'data_notificacao', 'paciente', 'sexo', 'idade',
       'mun_resid', 'mun_atend', 'rs', 'nome_exame', 'data_liberacao',
       'data_1o_sintomas']]
casos_novos_raw.to_excel('novos_casos_raw.xlsx', index=None)

In [9]:
obitos_novos_raw = obitos_novos_raw[['id', 'data_notificacao', 'paciente', 'sexo', 'idade',
       'mun_resid', 'mun_atend', 'rs', 'nome_exame', 'data_liberacao',
       'data_1o_sintomas','data_cura_obito']]
obitos_novos_raw.to_excel('novos_obitos_raw.xlsx', index=None)